In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'module')))
from openTable import *

import warnings
warnings.filterwarnings('ignore')

# import gensim
from gensim.models.ldamodel import LdaModel

# import spacy
from spacy.lang.id import Indonesian
nlp = Indonesian()  # use directly
stopwords = spacy.lang.id.stop_words.STOP_WORDS 
stopwords |= {"nya","jurusan","jurus","the","of"}

from json import loads
from ast import literal_eval
from pickle import load

from gensim import similarities

In [35]:
# #Akronim
def slang(tokenized_sentence):
    slang_word_dict = loads(open("slang_word_dict.txt", 'r').read())

    for index in range(len(tokenized_sentence)):
        for key, value in slang_word_dict.items():
            for v in value:
                if tokenized_sentence[index] == v:
                    tokenized_sentence[index] = key
                else:
                    continue
                    
    return " ".join(tokenized_sentence)

def preprocessing(text):
    text = re.sub('<[^<]+?>', '', str(text)) #remove tag
    text = text.lower() #lower\n",
    text = re.sub(r'[^a-z]',' ',str(text)) #get alphabet only
    text = re.sub(r'\s+', ' ', text) #remove white space
    text = re.sub(r'sobat pintar','',text) # sorry:(
    text = [token.text for token in nlp(text)] #Token
    text = slang(text)#slang word
    text = re.sub(r'\s+', ' ', text) #remove white space
    text = [token.lemma_ for token in nlp(text) if token.lemma_ not in stopwords] #Lemma & stopword
    
    return text

def load_model():
    #Load Model
    loaded_model = LdaModel.load('lda.h5')
    
    #Load Corpus
    file = open('corpus_LDA.pkl','rb')
    loaded_corpus = load(file)
    
    #Load Dict
    file = open('dictionary_LDA.gensim','rb')
    loaded_dict = load(file)
    
    return loaded_model,loaded_corpus,loaded_dict

def get_similar(entryId,n):
    loaded_model,loaded_corpus,loaded_dict = load_model()
    
    #Get Doc
    statement = " WHERE entryId = {}"
    data = open_table(['entryId','content'],'BlogsEntry',statement=statement.format(entryId))
    text = data[1].values[0]
    
    #Test new document
    bow = loaded_dict.doc2bow(preprocessing(text))
    
    lda_index = similarities.MatrixSimilarity(loaded_model[loaded_corpus])
    
    query = lda_index[loaded_model[bow]]
    # # Sort the similarities
    sort_sim = sorted(enumerate(query), key=lambda item: -item[1])
    
    result = [x[0] for x in sort_sim] #Get Univ ID
    
    return result[:n]

In [36]:
get_similar(204420,10)

[281, 64, 489, 67, 530, 601, 179, 640, 147, 349]